In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# read train.zip dataset and test1.zip dataset
import zipfile as Zip

Zip.ZipFile('/kaggle/input/dogs-vs-cats/train.zip').extractall()
Zip.ZipFile('/kaggle/input/dogs-vs-cats/test1.zip').extractall()

In [ ]:
# optimize train dataset
!cd '/kaggle/working/train'; mkdir dogs; mkdir cats

!cd '/kaggle/working/train'; ls | grep -e cat | xargs mv -t cats
!cd '/kaggle/working/train'; ls | grep -e dog | xargs mv -t dogs

In [ ]:
# check train dataset directory
!ls /kaggle/working/train -la

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

generator = ImageDataGenerator(rescale = 1/255., validation_split = 0.2)

train_gen = generator.flow_from_directory('/kaggle/working/train', target_size = (64, 64), batch_size = 16, class_mode = 'binary', subset = 'training')
valid_gen = generator.flow_from_directory('/kaggle/working/train', target_size = (64, 64), batch_size = 16, class_mode = 'binary', subset = 'validation')
test_gen = generator.flow_from_directory('/kaggle/working', target_size = (64, 64), batch_size = 16, classes = ['test1'])

In [ ]:
# check the shape
x_train, y_train = next(train_gen)
x_test, y_test = next(test_gen)
y_train = y_train.reshape((y_train.shape[0], 1))

x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
# build model
from keras.layers import *
from keras.models import *
from keras.applications import *
import numpy as np

def build_model():
    model = Sequential()
    model.add(VGG16(input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3]), include_top = False, weights = 'imagenet', pooling = 'max'))
    model.add(Dense(1024, activation = 'relu', kernel_initializer = 'he_normal'))
    model.add(Dense(512, activation = 'relu', kernel_initializer = 'he_normal'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation = 'sigmoid', kernel_initializer = 'he_normal'))

    return model

model = build_model()
model.summary()

In [ ]:
# if val_loss will not imporve consecutive 10 times(patience = 5), epochs will be stopped
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 5)
mc = ModelCheckpoint('check.h5', monitor = 'val_binary_accuracy', mode = 'max', verbose = 1, save_best_only=True)

In [ ]:
from keras.optimizers import SGD

model.compile(loss = 'binary_crossentropy', optimizer = SGD(lr=0.00002, momentum = 0.9, nesterov = True), metrics = ['binary_accuracy'])
history = model.fit(train_gen, epochs = 200, callbacks = [es, mc], batch_size = 8, steps_per_epoch = len(train_gen), validation_data = valid_gen)

In [ ]:
# check result on graph
import matplotlib.pyplot as plt

loss = history.history['loss']
acc = history.history['binary_accuracy']
val_loss = history.history['val_loss']
val_acc = history.history['val_binary_accuracy']

plt.plot(loss, label= 'loss')
plt.plot(acc, label= 'binary_accuracy')
plt.plot(val_loss, label= 'val_loss')
plt.plot(val_acc, label= 'val_binary_accuracy')

plt.legend()
plt.show()